In [ ]:
import py4dgeo
from sklearn import cluster
from scipy import spatial
import os
import numpy as np
import uuid
import datetime
from fourdgeo import utilities


def cluster_m3c2_changes(significant_changes, dbscan_eps, min_cluster_size):
    """
    Cluster M3C2 changes using DBSCAN and return clusters with their properties.
    :param significant_changes: Array of significant changes with shape (n, 4) where n is the number of points.
    :param dbscan_eps: The maximum distance between two samples for one to be considered as
    :param min_cluster_size: The minimum number of samples in a cluster.
    :return: A list of clusters with their properties.
    """
    # DBSCAN clustering
    dbscan = cluster.DBSCAN(eps=dbscan_eps, min_samples=min_cluster_size)
    labels = dbscan.fit_predict(significant_changes[:, :-1])

    # Combine results and check that the labels are unique
    all_changes_with_labels = np.column_stack((significant_changes, labels))

    # Remove noise points (label -1)
    all_changes_with_labels = all_changes_with_labels[all_changes_with_labels[:, -1] != -1]
    return all_changes_with_labels

def extract_geoObjects_from_clusters(all_changes_with_labels, endDateTime_, filename_0, filename_1):
    """
    Extract observations from clusters of M3C2 changes.
    :param all_changes_with_labels: Array of significant changes with labels.
    :param endDateTime_: The end date and time of the observation.
    :param filename_0: The filename of the first epoch.
    :param filename_1: The filename of the second epoch.
    :return: A list of observations with geo objects.
    """
    # Extract unique cluster IDs and their counts
    cluster_ids, cluster_count = np.unique(all_changes_with_labels[:, -1],return_counts=True)

    # If no clusters found, continue to the next file
    if len(cluster_ids) == 0:
        print(f"No clusters found between {filename_0} and {filename_1}.")
        return

    # backgroundImageData_ = XXX

    geoObjects_ = []

    # If clusters found, print the largest and smallest cluster sizes    
    for cluster_id in range(len(cluster_ids)):
        xyz = all_changes_with_labels[all_changes_with_labels[:, -1] == cluster_ids[cluster_id], :3]
        m3c2_distances = all_changes_with_labels[all_changes_with_labels[:, -1] == cluster_ids[cluster_id], -2]
        convex_hull = spatial.ConvexHull(xyz)
        volume = convex_hull.volume
        area = convex_hull.area
        surface_to_volume_ratio = area / volume if volume > 0 else float('inf')
        m3c2_mean_distance = np.mean(np.abs(m3c2_distances))
        vertices_of_hull = convex_hull.points[convex_hull.vertices]

        # Create an geo object based on the cluster for the observations
        dateTime_ = endDateTime_
        # Create a unique ID for the cluster
        id_ = uuid.uuid4().hex
        # Type is "unknown", as we don't apply any classifier here
        type_ = "unknown"

        customEntityData_ = {
            "X_centroid": np.mean(xyz[:, 0]),
            "Y_centroid": np.mean(xyz[:, 1]),
            "Z_centroid": np.mean(xyz[:, 2]),
            "m3c2_magnitude_abs_average_per_cluster": m3c2_mean_distance,
            "volume": volume,
            "surface_area": area,
            "surface_to_volume_ratio": surface_to_volume_ratio,
            "cluster_size_points": int(cluster_count[cluster_id]),
            }
        
        geometry_ = {
            "type": "Polygon",
            # "coordinates": XXXX # Use projected coordinates for the polygon
            "coordinates": [vertices_of_hull[:, :2].tolist()]  # Only use X and Y coordinates for the polygon
        }

        geoObjects_.append({
            "id": id_,
            "type": type_,
            "dateTime": dateTime_,
            "geometry": geometry_,
            "customEntityData": customEntityData_
        })
    return geoObjects_

In [8]:
# Handle file download/reading here
infolder = r"/media/william/3b2c79aa-05fe-4d08-88bc-3669754da823/Ronny/low_res_bad_attr_clipped"

m3c2_settings = {"cyl_radius":1,
                 "normal_radii":[1.0,],
                 "max_distance": 10.0,
                 "registration_error":0.025
                 }

#DBScan parameters
dbscan_eps = 1
min_cluster_size = 100

In [ ]:
observations = {"observations": []}

# Gather & sort only the .laz files
laz_files = sorted(f for f in os.listdir(infolder) if f.endswith('.laz'))

# Walk through each consecutive pair
for prev_fname, curr_fname in zip(laz_files, laz_files[1:]):
    prev_path = os.path.join(infolder, prev_fname)
    curr_path = os.path.join(infolder, curr_fname)

    startDateTime = utilities.iso_timestamp(prev_fname)
    endDateTime   = utilities.iso_timestamp(curr_fname)

    # Load point clouds
    epoch_0 = py4dgeo.read_from_las(prev_path)
    epoch_1 = py4dgeo.read_from_las(curr_path)

    # Compute M3C2
    m3c2 = py4dgeo.M3C2(
        epochs=(epoch_0, epoch_1),
        corepoints=epoch_0.cloud,
        cyl_radius=m3c2_settings["cyl_radius"],
        normal_radii=m3c2_settings["normal_radii"],
        max_distance=m3c2_settings["max_distance"],
        registration_error=m3c2_settings["registration_error"],
    )
    distances, uncertainties = m3c2.run()

    # Mask & stack only significant changes
    mask = np.abs(distances) >= uncertainties["lodetection"]
    if not mask.any():
        print(f"No significant changes between {prev_fname} → {curr_fname}")
        continue

    significant_pts = epoch_0.cloud[mask]
    significant_d  = distances[mask]
    changes = np.column_stack((significant_pts, significant_d))

    # Cluster & extract geoObjects
    labeled = cluster_m3c2_changes(changes, dbscan_eps, min_cluster_size)
    geoObjects = extract_geoObjects_from_clusters(labeled, endDateTime, prev_fname, curr_fname)

    observations["observations"].append({
        "backgroundImageData": {},
        "startDateTime": startDateTime,
        "endDateTime": endDateTime,
        "geoObjects": geoObjects,
    })

[2025-06-30 16:08:18][INFO] Reading point cloud from file '/media/william/3b2c79aa-05fe-4d08-88bc-3669754da823/Ronny/low_res_bad_attr_clipped/240825_220005.laz'
[2025-06-30 16:08:18][INFO] Reading point cloud from file '/media/william/3b2c79aa-05fe-4d08-88bc-3669754da823/Ronny/low_res_bad_attr_clipped/240825_230005.laz'
[2025-06-30 16:08:18][INFO] Building KDTree structure with leaf parameter 10
[2025-06-30 16:08:18][INFO] Building KDTree structure with leaf parameter 10
No clusters found between 240825_220005.laz and 240825_230005.laz.
[2025-06-30 16:08:23][INFO] Reading point cloud from file '/media/william/3b2c79aa-05fe-4d08-88bc-3669754da823/Ronny/low_res_bad_attr_clipped/240825_230005.laz'
[2025-06-30 16:08:23][INFO] Reading point cloud from file '/media/william/3b2c79aa-05fe-4d08-88bc-3669754da823/Ronny/low_res_bad_attr_clipped/240826_000005.laz'
[2025-06-30 16:08:23][INFO] Building KDTree structure with leaf parameter 10
[2025-06-30 16:08:23][INFO] Building KDTree structure with

In [10]:
observations

{'observations': [{'backgroundImageData': {},
   'startDateTime': '2024-08-25T22:00:05',
   'endDateTime': '2024-08-25T23:00:05',
   'geoObjects': None},
  {'backgroundImageData': {},
   'startDateTime': '2024-08-25T23:00:05',
   'endDateTime': '2024-08-26T00:00:05',
   'geoObjects': None},
  {'backgroundImageData': {},
   'startDateTime': '2024-08-26T00:00:05',
   'endDateTime': '2024-08-26T01:00:06',
   'geoObjects': [{'id': '20a604a801ec420db6d314368851928e',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[[-258.019, 158.27100000000002],
        [-258.015, 158.19400000000002],
        [-258.005, 158.187],
        [-258.005, 158.18800000000002],
        [-258.397, 158.428],
        [-258.177, 158.293],
        [-258.129, 158.151],
        [-258.677, 158.487],
        [-258.25100000000003, 158.225],
        [-258.298, 158.255],
        [-258.615, 158.356],
        [-258.32, 158.17600000000002],
        [-259.5